In [4]:
#import modules
import json
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv
from envs import env
import time
import re
import os
import datetime
from datetime import timedelta
import statistics

In [5]:
#import classes from ./ folder
import postgresdbAccess

In [7]:
#read fibonacci retracements  from json
with open('fibLvl.json') as file:
    fibLvl = json.load(file)

In [8]:
def corConsistency(lD,scale):
    lD[3] = lD[2] - timedelta(hours=scale)
    consideredTime = lD[1] >= lD[5]
    cV = consideredTime[0].corr(consideredTime[1])
    return cV

In [9]:
def backtest():
    #initiate empty open positions
    openPositions = {}
    #initiate postgresdb class
    postgres = postgresdbAccess.postgresAccess()
    #get unique simbol list
    sql = ("SELECT distinct symbol FROM " + dbTable + ";")
    uniqueSymbol = pd.DataFrame(postgres.sqlQuery(sql))
    uniqueSymbol = uniqueSymbol[0]
    #initiate empty percent bar
    percentBar = 0
    #loop over every symbol in symbol list
    for symbol in uniqueSymbol:
        #print progress
        print(percentBar, " / ", len(uniqueSymbol))
        percentBar = percentBar + 1
        #skip if symbol is not to be considered
        #null askprice is sorted out with crawler already
        if not (symbol.endswith(baseCurrency) and
               len(symbol) < 11):
            continue
        #check if open position was left over from last symbol and simulate close
        if openPositions:
            #prepare sql for close
            openPositions['resultPercent'] = ((row[4] - openPositions['askPrice']) / openPositions['askPrice']) * 100 - brokerFees * 2.55
            openPositions['stopId'] = row[0]
            closeSql = ("UPDATE " + dbTable + " SET" +
            " startId = '" + str(int(openPositions['startId'])) +
            "', fibLevel = '" + str(openPositions['fibLevel']) +
            "', midId = '" + str(int(openPositions['id'])) +
            "', corValue = '" + str(openPositions['corValue']) +
            "', takeProfit = '" + str(openPositions['takeProfit']) +
            "', stopLoss = '" + str(openPositions['stopLoss']) +  
            "', resultpercent = '" + str(openPositions['resultPercent']) +
            "', stopid = '" + str(openPositions['stopId']) +
            "' WHERE id = '" + str(int(openPositions['id'])) +
            "';")
            postgres.sqlUpdate(closeSql)
            openPositions = {}
        #query for every line for symbol
        sql = ("SELECT id, time, symbol, askprice, " +
               "bidprice, highprice, quotevolume, " +
               "pricechangepercent, weightedavgprice, openprice" +
            " FROM " + dbTable + " WHERE" +
            " symbol like '" + symbol + "' order by id;")
        print(sql)
        bigData = pd.DataFrame(postgres.sqlQuery(sql))
        bigData[0] = pd.to_numeric(bigData[0])
        bigData[1] = pd.to_datetime(bigData[1])
        bigData[3] = pd.to_numeric(bigData[3])
        bigData[6] = pd.to_numeric(bigData[6])
        bigData[7] = pd.to_numeric(bigData[7])
        bigData[8] = pd.to_numeric(bigData[8])
        bigData[9] = pd.to_numeric(bigData[9])
        #get start of timedelta
        bigData[5] = bigData[1] - timedelta(hours=33)
        #loop over every row
        for index, row in bigData.iterrows():
            before_start_date = bigData[1] <= row[5]
            if len(bigData.loc[before_start_date]) > 0:
                #get data for consideration
                after_start_date = bigData[1] >= row[5]
                before_end_date = bigData[1] <= row[1]
                between_two_dates = after_start_date & before_end_date
                fibDates = bigData.loc[between_two_dates]
                diff = fibDates[3].max() - fibDates[3].min()
                # calculate fibRetracements
                fibRetracement = pd.DataFrame(fibLvl)
                maxAsk = fibDates[3].max()
                for lvl in fibRetracement:
                    fibRetracement[1] =  maxAsk - diff * fibRetracement[0]
                    fibRetracement[2] = fibRetracement[1] * 0.9995
                    fibRetracement[3] = fibRetracement[1] * 1.0005
                #calculate corvalue
                corValue = fibDates[0].corr(fibDates[3])
                corValue1 = corConsistency(fibDates,15)
                corValue2 = corConsistency(fibDates,5)
                corValue3 = corConsistency(fibDates,1)
                #if an open position exists, check if it can be closed
                if openPositions:
                    if (row[3] < openPositions['stopLoss'] or
                        row[3] > openPositions['takeProfit']):
                        #prepare sql for close
                        openPositions['resultPercent'] = ((row[4] - openPositions['askPrice']) / openPositions['askPrice']) * 100 - brokerFees * 3.5
                        openPositions['stopId'] = row[0]
                        closeSql = ("UPDATE " + dbTable + " SET" +
                        " startId = '" + str(int(openPositions['startId'])) +
                        "', fibLevel = '" + str(openPositions['fibLevel']) +
                        "', midId = '" + str(int(openPositions['id'])) +
                        "', corValue = '" + str(openPositions['corValue']) +
                        "', takeProfit = '" + str(openPositions['takeProfit']) +
                        "', stopLoss = '" + str(openPositions['stopLoss']) +  
                        "', resultpercent = '" + str(openPositions['resultPercent']) +
                        "', stopid = '" + str(openPositions['stopId']) +
                        "', stdev = '" + str(openPositions['stDev']) +
                        "' WHERE id = '" + str(int(openPositions['id'])) +
                        "';")
                        #update database
                        postgres.sqlUpdate(closeSql)
                        #clear open position
                        openPositions = {}
                else:
                    #loop over considered fibonacciretracements
                    for i in [7]:
                        #check if buy requirements are met
                        if (corValue > 0 and
                            corValue1 >= 0 and
                            corValue2 >= 0 and
                            corValue3 >= 0 and
                            row[3] < fibRetracement[3][i] and
                            row[3] > fibRetracement[2][i]):
                            openPositions['startId'] = fibDates[0].min()
                            openPositions['id'] = row[0]
                            openPositions['bidPrice'] = row[4]
                            openPositions['fibLevel'] = i
                            openPositions['symbol'] = symbol
                            openPositions['askPrice'] = row[3]
                            openPositions['corValue'] = corValue
                            openPositions['stDev'] = statistics.stdev(fibDates[8])
                            openPositions['takeProfit'] = fibRetracement[3][i+2]
                            openPositions['stopLoss'] = fibRetracement[2][i-1]
    #close database connection
    postgres.databaseClose()

In [11]:
testTables = [{"table":"backtesting","currency":"BNB"},
             {"table":"backtesting","currency":"BTC"},
             {"table":"backtesting","currency":"ETH"},
             {"table":"backtesting","currency":"ADA"}]
for iteration in testTables:
    load_dotenv('../.env')
    try:
        liveVolume=env("liveVolume")
        apiSecret=env('apiSecret')
        apiKey=env('apiKey')
        dbTable=iteration["table"]
        baseCurrency=iteration["currency"]
        brokerFees=float(env('brokerFees'))
        liveTrading=env("liveTrading", 'False').lower() in ('true', '1', 't')
    except KeyError:
        print("No env variables set.")
        sys.exit(1)
    client = Client(apiKey, apiSecret, {'timeout':600})
    backtest()

0  /  752
1  /  752
2  /  752
3  /  752
4  /  752
5  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ADXBNB' order by id;


TypeError: unsupported operand type(s) for -: 'str' and 'Timedelta'